Testing data positive feedback dynamic (20%)

In [36]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [43]:
user_following = np.load('../Data/npy/user_following_1489.npy')[:100]
all_3374 = np.load('../Data/npy/all_3374D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')[:100]
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')[:100]

In [46]:
#The number of followers for each YouTuber
YouTuber_followers = np.sum(user_following, axis=0)
print(YouTuber_followers)
over5 = 0
for num in YouTuber_followers:
    if num >= 5:
        over5+=1
print('The num of followers over 5:',over5)

[ 8  0  2 24  1  0  0 11 47  7 25 32 10  1 16  1  6 24  3  2  1  6  4  0
 39 39 10 16 27 22  0  2 17 15  0  2  0  8  2  1 28 16  3 33  9 24  0  3
  0  9  0  0  0  2 28  2  2 18  0  3 23  0  2 39  0  1  1 65  0  2 18 21
  7  1 47 14 19  1  3 66  3  0 28  3 20  0  6  0]
The num of followers over 5: 42


In [47]:
print('user_following shape ',user_following.shape)
print('all_3374 shape ',all_3374.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (100, 88)
all_3374 shape  (88, 3374)
user_category shape  (100, 17)
YouTuber_category shape  (88, 17)


In [48]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (100, 17)


In [49]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)

In [50]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  21


In [51]:
over_10 = 0
over_8 = 0
over_12 = 0
user_idx_over10 = []
avg_over10 = 0
for i in range(len(num_of_follower)):
    num = num_of_follower[i]
    if num >= 10:
        over_10 += 1
        user_idx_over10.append(i)
        avg_over10+=num
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)
print('avg 10:',avg_over10/len(user_idx_over10))

over 8: 68
over 10: 56
over 12: 28
avg 10: 12.517857142857142


# Training data and Testing data

In [52]:
test_amount = 10
yt_test_amount = 20

In [53]:
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(3)
test_idx = random.sample(user_idx,test_amount)

In [54]:
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

5 3
2
6 3
3
14 7
7
11 6
5
5 3
2
6 3
3
7 4
3
11 6
5
10 5
5
9 5
4


In [55]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
avg

4.5

In [56]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 100
The length of train_f: 100
The length of test_t: 10
The length of test_f: 10


# Recommendation  Module

In [57]:
n = len(user_following)
m = 88  
k = 128
l = 3374
embedding_dims = 150

In [83]:
def training(save_name): 
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()

    #use_true=init_list_of_objects(136)
    #use_test=init_list_of_objects(136)

    #train_pair_t=[] #positive feedback
    #train_pair_f=[] #negative feedback
    train_yes_id=[] 
    for q in range(10):
        #clear_output()
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yes=[]
            yesr=[]
        
        
            sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            train_yes_id.append(sample) #sample全部丟進去
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            #change
            r_3=np.zeros(len(sample)) 
         
            for b in range(len(sample)):
                yes.append(all_3374[sample[b]])
                yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
                #print('YouTuber_category ', YouTuber_category[sample[k]])
                #print('User_category ',user_category_norm[z])
            #print(yesr)
        
            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)
        
            yes=np.array(yes)
        
            #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                #print(ta,'--> positive feedback')
            
                pos = sample.index(ta)
                #new_sample = np.delete(sample,[pos])
                #new_yes = np.delete(yes,[pos],axis=0)
                #new_r_3 = np.delete(r_3,[pos])
                new_sample = sample
                new_yes = yes
                new_r_3 = r_3
                #print(len(yes),len(new_yes))
                #print(yes)
                #print(new_yes)
            
            
            
                #ta=random.choice(train_t[z]) #ta is true positve photo
                #train_pair_t.append(ta)
                image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
                #print('Image_1 shape ',image_1.shape)
                #train_f_sample = random.sample(train_f[z],20)
                #print('True:',train_t_sample,'Now:',ta)
                #print('False:',train_f_sample)
                for b in train_f_sample:
                    #print('likes:',ta,';Not likes:',b)
                    #b=random.choice(train_f[z])  #b is no feedback photo
                    #train_pair_f.append(b)
                    image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                    #print('Image_2 shape',image_2.shape)
            
                    #use_test[z].append(b)
                    _last_be_relu,_norm_par,_a_list,r3,_auc, _loss,_=sess.run([last_be_relu,norm_par,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: new_yes , l_id:new_sample, l_id_len:[len(new_sample)],r:new_r_3,
                                        image_i:image_1,image_j:image_2})
                
                    #print(XUIJ)
                    #print('loss=',_loss)
                    #print('auc=',_auc)
                    #print('user positive negative',z,ta,b)
                    #for each_par in _norm_par:
                    #    print(each_par.shape)
                    #    print(each_par)
                    #print('be_relu,wu,wy,wa,wv',_last_be_relu)
                    #print('alpha softmax:',r3)
                    #print('before softmax:',_a_list)
                    #print('---------------------------------------------------')
        
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
                #now1+=1
        
            #np.savez('../Data/latent_factor/YRM_up10/'+str(q)+'_'+str(z)+'.npz', User=U_history, YouTuber=Y_history)
    
        #print('mine:',xuij_auc/136)    
        #print('a_list_soft:',r3)
        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
    #print('Total cost ',time.time()-t0,' sec')   
    U, Y, A, E, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])
    np.savez('../Data/grid_search_weight/new_dims/'+save_name+'.npz', 
                        U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)
    return U, Y, A, E, Au, Ay, Aa, Av,B

In [86]:
par_weights = [0.01,0.001,0.0001,0.00001]
beta_weights = [1,0.1,0.01]
Embedding_weights = [1,0.1,0.01]
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                print(paru_weight,pary_weight,beta_weight,Embedding_weight,':START!!---------------')
                #clear_output()
                """
                n: the number of users
                m: the number of YouTubers
                k: latent dims
                l: feature dims
                """
                tf.reset_default_graph()

                user = tf.placeholder(tf.int32,shape=(1,))
                i = tf.placeholder(tf.int32, shape=(1,))
                j = tf.placeholder(tf.int32, shape=(1,))

                #多少個auxliary 
                xf = tf.placeholder(tf.float32, shape=(None,l))
                l_id = tf.placeholder(tf.int32, shape=(None,))
                l_id_len = tf.placeholder(tf.int32,shape=(1,))
                r = tf.placeholder(tf.float32,shape=(None,))


                image_i = tf.placeholder(tf.float32, shape=(1,l))
                image_j = tf.placeholder(tf.float32, shape=(1,l))

                with tf.variable_scope("item_level"):
                    user_latent = tf.get_variable("user_latent", [n, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                    item_latent = tf.get_variable("item_latent", [m, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                    aux_item = tf.get_variable("aux_item", [m, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                    Wu = tf.get_variable("Wu", [n,m,k],  #n*m*kt
                                                      initializer=tf.contrib.layers.xavier_initializer())
                    Wy = tf.get_variable("Wy", [n,m,k],  #1*k 每個user都有一個wy
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Wa = tf.get_variable("Wa", [n,m,k],  #1*k
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Wv = tf.get_variable("Wv", [n,m,l],  #1*l
                                                    initializer=tf.contrib.layers.xavier_initializer())

                    #每個user 對於每個YouTuber都有一個權重
                    #w1拿掉，wu
                    #hyper?
                    
                    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                    ########## Error part, how to get auxisize dynamically
                    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                with tf.variable_scope('feature_level'):
                    embedding = tf.get_variable("embedding", [embedding_dims,l],
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Beta = tf.get_variable("beta", [n,embedding_dims],
                                             # initializer=tf.contrib.layers.xavier_initializer())
                                                     initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                #lookup the latent factors by user and id
                u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                vi = tf.nn.embedding_lookup(item_latent, i) 
                vj = tf.nn.embedding_lookup(item_latent, j)
                
                
                wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                a_list=tf.Variable([])
                q = tf.constant(0)
                def att_cond(q,a_list):
                    return tf.less(q,l_id_len[0])
                def att_body(q,a_list):
                    xfi = tf.expand_dims(xf[q],0) #(1,l)
                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0)
                    a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                            tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                            tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                            tf.matmul(wvui, xfi, transpose_b=True))[0][0])*r[q]]],0)
                    q += 1
                    return q,  a_list
                _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                a_list_soft=tf.nn.softmax(a_list)
                
                norm_par = [wu,wy,wa,wv]
                
                wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                wv_be_relu = tf.matmul(wvui, tf.expand_dims(xf[-1],0), transpose_b=True)
                last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]
                                       
                aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                q = tf.constant(0)
                def sum_att_cond(q,aux_np):
                    return tf.less(q,l_id_len[0])

                def sum_att_body(q,aux_np):
                    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                    q += 1
                    return q, aux_np

                _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                """
                for q in range(3): #取q個auxliary item
                    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                """

                aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                #tf.print('aux attention:',aux_np)
                aux_np+=u #user_latent factor + sum (alpha*auxilary)
                aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                #矩陣中對應函數各自相乘
                # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                xuij = tf.subtract(xui,xuj)

        
                l2_norm = tf.add_n([
                            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                            paru_weight * tf.reduce_sum(tf.multiply(wu, wu)),
                            pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                            pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                            pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                            beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                            Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                          ])

                loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                auc = tf.reduce_mean(tf.to_float(xuij > 0))
                
                Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(str(pary_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight))
                #np.savez('../Data/grid_search_weight/new_dims/'+str(pary_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight)+'.npz', 
                #         U=Ur, Y=Yr, A=Ar, E=Er, Wu=Aur, Wy=Ayr, Wa=Aar, Wv=Avr,B=Br)
                print('Finish,',str(paru_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight))

0.01 0.01 1 1 :START!!---------------
Iteraction: 0
total_loss:----------------- [[2.401668]]
train_auc:------------------- 0.9595188284518829
time: 172.1864197254181  sec
Iteraction: 1
total_loss:----------------- [[0.47301975]]
train_auc:------------------- 0.9815899581589959
time: 344.4140074253082  sec
Iteraction: 2
total_loss:----------------- [[0.41074175]]
train_auc:------------------- 0.9866108786610879
time: 516.2560789585114  sec
Iteraction: 3
total_loss:----------------- [[0.3410428]]
train_auc:------------------- 0.9878138075313807
time: 688.1078984737396  sec
Iteraction: 4
total_loss:----------------- [[0.28813824]]
train_auc:------------------- 0.9836297071129707
time: 860.0533907413483  sec
Iteraction: 5
total_loss:----------------- [[0.25065586]]
train_auc:------------------- 0.9791841004184101
time: 1032.0190222263336  sec
Iteraction: 6
total_loss:----------------- [[0.22590321]]
train_auc:------------------- 0.9776150627615062
time: 1203.7753105163574  sec
Iteraction:

total_loss:----------------- [[0.14347523]]
train_auc:------------------- 0.9848326359832636
time: 1546.840432882309  sec
Iteraction: 9
total_loss:----------------- [[0.13478759]]
train_auc:------------------- 0.9869246861924686
time: 1718.609989643097  sec
Finish, 0.01_0.01_0.1_0.01
0.01 0.01 0.01 1 :START!!---------------
Iteraction: 0
total_loss:----------------- [[2.2783098]]
train_auc:------------------- 0.9583682008368201
time: 173.1306676864624  sec
Iteraction: 1
total_loss:----------------- [[0.27166703]]
train_auc:------------------- 0.9760460251046025
time: 345.8827545642853  sec
Iteraction: 2
total_loss:----------------- [[0.24005799]]
train_auc:------------------- 0.9779811715481171
time: 518.6777076721191  sec
Iteraction: 3
total_loss:----------------- [[0.2183634]]
train_auc:------------------- 0.9807008368200837
time: 691.5116851329803  sec
Iteraction: 4
total_loss:----------------- [[0.20963679]]
train_auc:------------------- 0.9816422594142259
time: 864.1979472637177  

total_loss:----------------- [[0.19969806]]
train_auc:------------------- 0.9835251046025104
time: 1204.782915353775  sec
Iteraction: 7
total_loss:----------------- [[0.19145289]]
train_auc:------------------- 0.9845188284518829
time: 1376.8189771175385  sec
Iteraction: 8
total_loss:----------------- [[0.17963152]]
train_auc:------------------- 0.9849372384937238
time: 1548.9640889167786  sec
Iteraction: 9
total_loss:----------------- [[0.16566473]]
train_auc:------------------- 0.9869769874476988
time: 1721.1128549575806  sec
Finish, 0.01_0.001_1_0.01
0.01 0.001 0.1 1 :START!!---------------
Iteraction: 0
total_loss:----------------- [[2.2988248]]
train_auc:------------------- 0.9575836820083682
time: 171.19646668434143  sec
Iteraction: 1
total_loss:----------------- [[0.3415696]]
train_auc:------------------- 0.9772489539748954
time: 342.19838404655457  sec
Iteraction: 2
total_loss:----------------- [[0.31775048]]
train_auc:------------------- 0.9793410041841004
time: 513.22243952751

total_loss:----------------- [[0.14570868]]
train_auc:------------------- 0.9736924686192469
time: 861.394740819931  sec
Iteraction: 5
total_loss:----------------- [[0.13533208]]
train_auc:------------------- 0.9780857740585774
time: 1033.6327812671661  sec
Iteraction: 6
total_loss:----------------- [[0.13670151]]
train_auc:------------------- 0.9766213389121339
time: 1205.7475752830505  sec
Iteraction: 7
total_loss:----------------- [[0.12693769]]
train_auc:------------------- 0.980805439330544
time: 1377.8576803207397  sec
Iteraction: 8
total_loss:----------------- [[0.12121915]]
train_auc:------------------- 0.9819560669456067
time: 1549.9632542133331  sec
Iteraction: 9
total_loss:----------------- [[0.11192633]]
train_auc:------------------- 0.983734309623431
time: 1722.1845738887787  sec
Finish, 0.01_0.001_0.01_0.01
0.01 0.0001 1 1 :START!!---------------
Iteraction: 0
total_loss:----------------- [[2.3938453]]
train_auc:------------------- 0.9617154811715481
time: 172.42631292343

total_loss:----------------- [[0.19738719]]
train_auc:------------------- 0.9661087866108786
time: 516.5474398136139  sec
Iteraction: 3
total_loss:----------------- [[0.18535373]]
train_auc:------------------- 0.9712343096234309
time: 687.6913080215454  sec
Iteraction: 4
total_loss:----------------- [[0.17952071]]
train_auc:------------------- 0.9751569037656904
time: 858.8258008956909  sec
Iteraction: 5
total_loss:----------------- [[0.16162066]]
train_auc:------------------- 0.9801778242677824
time: 1030.0006051063538  sec
Iteraction: 6
total_loss:----------------- [[0.15871863]]
train_auc:------------------- 0.980805439330544
time: 1201.132029056549  sec
Iteraction: 7
total_loss:----------------- [[0.14696708]]
train_auc:------------------- 0.9849372384937238
time: 1372.0939102172852  sec
Iteraction: 8
total_loss:----------------- [[0.14609575]]
train_auc:------------------- 0.9857217573221757
time: 1543.192709684372  sec
Iteraction: 9
total_loss:----------------- [[0.14096059]]
tra

Finish, 0.01_1e-05_1_0.1
0.01 1e-05 1 0.01 :START!!---------------
Iteraction: 0
total_loss:----------------- [[0.3707007]]
train_auc:------------------- 0.9346234309623431
time: 173.56282949447632  sec
Iteraction: 1
total_loss:----------------- [[0.2905917]]
train_auc:------------------- 0.9555962343096235
time: 346.855256319046  sec
Iteraction: 2
total_loss:----------------- [[0.26916888]]
train_auc:------------------- 0.9660564853556486
time: 520.0999524593353  sec
Iteraction: 3
total_loss:----------------- [[0.24663365]]
train_auc:------------------- 0.9725418410041841
time: 693.4170715808868  sec
Iteraction: 4
total_loss:----------------- [[0.23221883]]
train_auc:------------------- 0.9760460251046025
time: 866.7879745960236  sec
Iteraction: 5
total_loss:----------------- [[0.21027508]]
train_auc:------------------- 0.9803870292887029
time: 1040.1018941402435  sec
Iteraction: 6
total_loss:----------------- [[0.20203964]]
train_auc:------------------- 0.982897489539749
time: 1213.3

total_loss:----------------- [[0.14001273]]
train_auc:------------------- 0.9857217573221757
time: 1549.4606258869171  sec
Iteraction: 9
total_loss:----------------- [[0.14726007]]
train_auc:------------------- 0.9834728033472804
time: 1721.577942609787  sec
Finish, 0.01_1e-05_0.01_0.1
0.01 1e-05 0.01 0.01 :START!!---------------
Iteraction: 0
total_loss:----------------- [[0.30458924]]
train_auc:------------------- 0.9413179916317992
time: 172.30079650878906  sec
Iteraction: 1
total_loss:----------------- [[0.18889277]]
train_auc:------------------- 0.9627092050209205
time: 344.24811601638794  sec
Iteraction: 2
total_loss:----------------- [[0.17118275]]
train_auc:------------------- 0.968723849372385
time: 516.2776250839233  sec
Iteraction: 3
total_loss:----------------- [[0.16243395]]
train_auc:------------------- 0.9700313807531381
time: 688.3180012702942  sec
Iteraction: 4
total_loss:----------------- [[0.1505895]]
train_auc:------------------- 0.9733263598326359
time: 860.4137010

KeyboardInterrupt: 

In [ ]:
def testing(U, Y, A, E, A1, Au, Ay, Aa, Av,B):
    #print(A1)
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    max1 = 0
    maxu = 0
    maxy = 0
    maxa = 0
    maxv = 0
    min1 = 100000000000000000
    minu = 100000000000000000
    miny = 100000000000000000
    mina = 100000000000000000
    minv = 100000000000000000
    test_yes_id=[]
    for s in range(test_amount):
        print(s,test_idx[s])

        yes=[]
        sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        #sample=result_yes_id[now]
        test_yes_id.append(sample)
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            #print(test_idx[s])
            #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))

            #Observe each part in attention , below par are all (128,1)
            testW1 = np.sum(A1[test_idx[s]])
            #print(A1[test_idx[s]])
            WuUu = np.sum(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T))
            WyYy = np.sum(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T))
            WaAa = np.sum(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T))
            WvVy = np.sum(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))
            print('The sum of each par -->\nw1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
            if testW1 > max1:
                max1 = testW1
            if testW1 < min1:
                min1 = testW1
            if WuUu > maxu:
                maxu = WuUu
            if WuUu < minu:
                minu = WuUu
            if WyYy > maxy:
                maxy = WyYy
            if WyYy < miny:
                miny = WyYy
            if WaAa > maxa:
                maxa = WaAa
            if WaAa < mina:
                mina = WaAa
            if WvVy > maxv:
                mxv = WvVy
            if WvVy < minv:
                minv = WvVy
            #Have w1
            #alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
            #                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
            #Without w1
            alpha[a]=np.sum((relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                                np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
        mul=np.zeros((1,128))
        print('alpha------------',alpha)
        print('softmax alpha--------------',softmax(alpha))
        for i in range(len(sample)):
            mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
    #print(max1,maxu,maxy,maxa,maxv)
    #print(min1,minu,miny,mina,minv)
    return RS

In [ ]:
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/0105/'
files = os.listdir(path)
#U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])
for file in files:
    par_data = np.load(path+file)
    U = par_data['U']
    Y = par_data['Y']
    A = par_data['A']
    E = par_data['E']
    W1 = par_data['W1']
    Wu = par_data['Wu']
    Wy = par_data['Wy']
    Wa = par_data['Wa']
    Wv = par_data['Wv']
    B = par_data['B']
    RS = testing(U, Y, A, E, W1, Wu, Wy, Wa, Wv,B)

# Get latent factor and Each weight

In [ ]:
U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])

In [ ]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)
print('Embedding shape:',E.shape)

In [71]:
U, Y, A, E, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])

NameError: name 'sess' is not defined

# Testing Part

In [ ]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        
        #Observe each part in attention , below par are all (128,1)
        testW1 = np.sum(np.multiply(A1[test_idx[s]],A1[test_idx[s]]))
        WuUu = np.sum(np.multiply(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T),np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)))
        WyYy = np.sum(np.multiply(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T),np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)))
        WaAa = np.sum(np.multiply(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T),np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T)))
        WvVy = np.sum(np.multiply(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T),np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T)))
        print('w1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
        
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
#print(RS[s])

In [ ]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [ ]:
target

In [ ]:
testRS

In [ ]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

In [ ]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [ ]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
print('avg_accuarcy for count_0:',avg_acc)
print(acc_0)

In [ ]:
acc_0

In [ ]:
total

# Top 1 

In [ ]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [ ]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

In [ ]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

In [ ]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

# Top 3

In [ ]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [ ]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

In [ ]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

# Top 5

In [ ]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

In [ ]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

In [ ]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

In [ ]:
#User_latent_factor = loaddata['User']
#YouTuber_latent_factor = loaddata['YouTuber']

In [ ]:
np.savez('../Data/latent_factor/YRM_up10_ALL/Final/1226.npz', User=U, YouTuber=Y)

In [ ]:
def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [ ]:
softmax([-0.000000000000000000000000000000000000001,0.00000000000000000000000000000000000001])

stuff
ddkdkk


stuff
